In [1]:
#from __future__ import print_function
import argparse
import os
import imp
import algorithms as alg
from dataloader import DataLoader, GenericDataset

In [2]:
import torch
import torch.nn as nn
torch.cuda.device_count()

1L

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
args_opt = Namespace(
    exp = 'Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64',
    evaluate = False,
    checkpoint = 3,
    num_workers = 1,
    cuda = True,
    disp_step= 5
)

In [5]:
exp_config_file = os.path.join('.','config',args_opt.exp+'.py')
# if args_opt.semi == -1:
exp_directory = os.path.join('.','experiments',args_opt.exp)
# else:
#    assert(args_opt.semi>0)
#    exp_directory = os.path.join('.','experiments/unsupervised',args_opt.exp+'_semi'+str(args_opt.semi))

In [6]:
exp_config_file

'./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64.py'

In [7]:
# Load the configuration params of the experiment
print('Launching experiment: %s' % exp_config_file)
config = imp.load_source("",exp_config_file).config
config['exp_dir'] = exp_directory # the place where logs, models, and other stuff will be stored
print("Loading experiment %s from file: %s" % (args_opt.exp, exp_config_file))
print("Generated logs, snapshots, and model files will be stored on %s" % (config['exp_dir']))

Launching experiment: ./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64.py
Loading experiment Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64 from file: ./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64.py
Generated logs, snapshots, and model files will be stored on ./experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64


In [8]:
# Set train and test datasets and the corresponding data loaders
data_train_opt = config['data_train_opt']
data_test_opt = config['data_test_opt']
num_imgs_per_cat = data_train_opt['num_imgs_per_cat'] if ('num_imgs_per_cat' in data_train_opt) else None

In [9]:
dataset_train = GenericDataset(
    dataset_name=data_train_opt['dataset_name'],
    split=data_train_opt['split'],
    random_sized_crop=data_train_opt['random_sized_crop'],
num_imgs_per_cat=num_imgs_per_cat)

custom
num_imgs_per_category 64


In [10]:
dataset_test = GenericDataset(
    dataset_name=data_test_opt['dataset_name'],
    split=data_test_opt['split'],
    random_sized_crop=data_test_opt['random_sized_crop'])

custom


In [11]:
dloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=data_train_opt['batch_size'],
    unsupervised=data_train_opt['unsupervised'],
    epoch_size=data_train_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=True)

In [12]:
dloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=data_test_opt['batch_size'],
    unsupervised=data_test_opt['unsupervised'],
    epoch_size=data_test_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=False)

In [13]:
len(dloader_train.dataset)

64000

In [14]:
config['disp_step'] = args_opt.disp_step

In [15]:
algorithm = getattr(alg, config['algorithm_type'])(config)

2019-05-15 02:29:02,737 - algorithms.Algorithm - INFO   - Algorithm options {'max_num_epochs': 40, 'disp_step': 5, 'algorithm_type': 'FeatureClassificationModel', 'criterions': {'loss': {'opt': None, 'ctype': 'CrossEntropyLoss'}}, 'data_test_opt': {'unsupervised': False, 'batch_size': 150, 'epoch_size': None, 'random_sized_crop': False, 'split': 'supervised/val', 'dataset_name': 'custom'}, 'exp_dir': './experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64', 'out_feat_keys': ['conv5'], 'networks': {'classifier': {'opt': [{'cls_type': 'Alexnet_conv5', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': None}, 'feat_extractor': {'opt': {'num_classes': 4}, 'def_file': 'architectures/AlexNet.py', 'optim_params': None, 'pretrained': './

In [16]:
# device = torch.device("cuda")
# device

In [17]:
if args_opt.cuda: # enable cuda
#     if torch.cuda.device_count() > 1:
#         algorithm = nn.DataParallel(algorithm)
    algorithm.load_to_gpu()
#     algorithm.to(device)
if args_opt.checkpoint > 0: # load checkpoint
    algorithm.load_checkpoint(args_opt.checkpoint, train= (not args_opt.evaluate))

2019-05-15 02:29:22,421 - algorithms.Algorithm - INFO   - Load checkpoint of epoch 3
2019-05-15 02:29:23,609 - algorithms.Algorithm - INFO   - Initialize optimizer: sgd with params: {'LUT_lr': [(10, 0.01), (20, 0.002), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.1, 'weight_decay': 0.0005, 'momentum': 0.9} for netwotk: classifier


In [ ]:
if not args_opt.evaluate: # train the algorithm
    algorithm.solve(dloader_train, dloader_test)
else:
    algorithm.evaluate(dloader_test) # evaluate the algorithm


2019-05-15 02:29:24,822 - algorithms.Algorithm - INFO   - Training epoch [  4 /  40]
2019-05-15 02:29:24,825 - algorithms.Algorithm - INFO   - ==> Set to classifier optimizer lr = 0.0100000000
2019-05-15 02:29:24,826 - algorithms.Algorithm - INFO   - Training: Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64
 37%|███▋      | 159/427 [12:27<20:59,  4.70s/it]2019-05-15 02:41:56,707 - algorithms.Algorithm - INFO   - ==> Iteration [  4][ 160 /  426]: {'process_time': 68.091, 'prec5_c1': 57.2167, 'loss': 3.3461, 'prec1_c1': 33.2542, 'load_time': 31.909}


 77%|███████▋  | 329/427 [25:12<07:09,  4.38s/it]2019-05-15 02:54:41,194 - algorithms.Algorithm - INFO   - ==> Iteration [  4][ 330 /  426]: {'process_time': 66.3818, 'prec5_c1': 57.0222, 'loss': 3.3459, 'prec1_c1': 33.0101, 'load_time': 33.6182}


100%|██████████| 427/427 [32:34<00:00,  5.30s/it]
2019-05-15 03:01:59,540 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 65.7834, 'prec5_c1': 56.7752, 'loss': 3.3458, 'prec1_c1': 32.808, 'load_time': 34.2166}
2019-05-15 03:02:00,859 - algorithms.Algorithm - INFO   - Evaluating: Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64
2019-05-15 03:02:00,865 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [24:06<00:00,  2.43s/it]
2019-05-15 03:26:07,254 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 54.1057, 'prec5_c1': 42.6151, 'loss': 4.0221, 'prec1_c1': 20.9118, 'load_time': 45.8943}
2019-05-15 03:26:07,280 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 54.1057, 'prec5_c1': 42.6151, 'loss': 4.0221, 'prec1_c1': 20.9118, 'load_time': 45.8943}
2019-05-15 03:26:07,281 - algorithms.Algorithm - INFO   - Training epoch [  5 /  40]
2019-05-15 03:26:07,283 - a

 51%|█████▏    | 219/427 [17:34<16:27,  4.75s/it]2019-05-15 03:43:47,068 - algorithms.Algorithm - INFO   - ==> Iteration [  5][ 220 /  426]: {'process_time': 69.6547, 'prec5_c1': 63.3424, 'loss': 3.01, 'prec1_c1': 38.8485, 'load_time': 30.3453}


 91%|█████████ | 389/427 [31:09<03:12,  5.06s/it]2019-05-15 03:57:21,566 - algorithms.Algorithm - INFO   - ==> Iteration [  5][ 390 /  426]: {'process_time': 67.4253, 'prec5_c1': 62.7675, 'loss': 3.0304, 'prec1_c1': 38.1419, 'load_time': 32.5747}


...

 67%|██████▋   | 284/427 [21:27<11:21,  4.76s/it]2019-05-15 07:18:04,275 - algorithms.Algorithm - INFO   - ==> Iteration [  9][ 285 /  426]: {'process_time': 60.9136, 'prec5_c1': 79.6561, 'loss': 2.0824, 'prec1_c1': 57.0456, 'load_time': 39.0864}


100%|██████████| 427/427 [32:48<00:00,  4.16s/it]
2019-05-15 07:29:20,110 - algorithms.Algorithm - INFO   - ==> Training stats: {'process_time': 64.7824, 'prec5_c1': 78.9313, 'loss': 2.1227, 'prec1_c1': 55.7658, 'load_time': 35.2176}
2019-05-15 07:29:21,085 - algorithms.Algorithm - INFO   - Evaluating: Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features_64
2019-05-15 07:29:21,087 - algorithms.Algorithm - INFO   - ==> Dataset: custom_supervised/val [64000 images]
100%|██████████| 427/427 [22:18<00:00,  2.62s/it]
2019-05-15 07:51:39,766 - algorithms.Algorithm - INFO   - ==> Results: {'process_time': 53.036, 'prec5_c1': 46.242, 'loss': 3.8451, 'prec1_c1': 23.9844, 'load_time': 46.964}
2019-05-15 07:51:39,785 - algorithms.Algorithm - INFO   - ==> Evaluation stats: {'process_time': 53.036, 'prec5_c1': 46.242, 'loss': 3.8451, 'prec1_c1': 23.9844, 'load_time': 46.964}
2019-05-15 07:51:39,786 - algorithms.Algorithm - INFO   - Training epoch [ 10 /  40]
2019-05-15 07:51:39,787 - algori

 41%|████      | 174/427 [14:42<22:36,  5.36s/it]2019-05-15 08:06:28,332 - algorithms.Algorithm - INFO   - ==> Iteration [ 10][ 175 /  426]: {'process_time': 72.0021, 'prec5_c1': 83.2114, 'loss': 1.8752, 'prec1_c1': 61.6152, 'load_time': 27.9979}


 81%|████████  | 344/427 [30:51<08:31,  6.17s/it]2019-05-15 08:22:39,462 - algorithms.Algorithm - INFO   - ==> Iteration [ 10][ 345 /  426]: {'process_time': 72.7481, 'prec5_c1': 82.3188, 'loss': 1.9189, 'prec1_c1': 60.3498, 'load_time': 27.2519}


 90%|████████▉ | 383/427 [34:53<04:18,  5.88s/it]

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torchvision/transforms/functional.py", line 48, in to_tensor